In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from pycoingecko import CoinGeckoAPI

In [ ]:
# Configuration
TOP_N = 10
LOOKBACK_DAYS = 180
RETURN_HORIZON = 5
RANDOM_STATE = 42

In [ ]:
def fetch_top_coins_by_market_cap(n = TOP_N):
    """
    Fetches top N crytocurrencies by market cap from CoinGecko API
    """

    cg = CoinGeckoAPI()
    coins = cg.get_coins_markets(vs_currency = 'usd', order = 'market_cap_desc',
                                 per_page = n, page = 1, sparkline = False)
    return [coin['id'] for coin in coins]


In [ ]:
def fetch_historical_data(coin_id, days = LOOKBACK_DAYS):
    """
    Fetch historical OHLCV data for a single coin
    """

    cg = CoinGeckoAPI()
    data = cg.get_coin_market_chart_by_id(id = coin_id, vs_currency = 'usd', days = days)

    df = pd.DataFrame({
        'timestamp': [x[0] for x in data['prices']],
        'price': [x[1] for x in data['prices']],
        'market_cap': [x[1] for x in data['market_caps']],
        'volume': [x[0] for x in data['total_volumes']],
    })

    df['Date'] = pd.to_datetime(df['timestamp'], unit = 'ms')
    df = df.drop('timestamp', axis = 1)
    df = df.rename(columns = {'price': 'Close', 'volume': 'Volume', 'market_cap': 'MarketCap'})
    df['Ticker'] = coin_id

    return df[['Date', 'Ticker', 'Close', 'Volume', 'MarketCap']]

In [ ]:
def build_crypto_dataset(top_n = TOP_N, days = LOOKBACK_DAYS):
    """
    Build comprehensive dataset for top N crypto assets
    """
    
    coin_ids = fetch_top_coins_by_market_cap(top_n)
    print(f"Coins selected: {coin_ids}")

    all_data = []
    for i, coin_id in enumerate(coin_ids):
        try:
            df = fetch_historical_data(coin_id, days)
            all_data.append(df)
        except Exception as e:
            print(f"Error fetching {coin_id}: {e}")
            continue

    data = pd.concat(all_data, ignore_index = True)
    print(f"Total records fetched: {len(data)}")
    return data

SyntaxError: incomplete input (4102869004.py, line 2)